In [123]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [125]:
from src.inference import get_feature_store

In [126]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1) ),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 00:08:21,515 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:08:21,521 INFO: Initializing external client
2025-03-06 00:08:21,522 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:08:22,318 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669
Fetching data from 2025-02-05 05:08:21.515796+00:00 to 2025-03-06 04:08:21.515796+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.94s) 


In [127]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 06:00:00+00:00,2,0
1,2025-02-05 07:00:00+00:00,2,0
2,2025-02-05 08:00:00+00:00,2,0
3,2025-02-05 09:00:00+00:00,2,0
4,2025-02-05 10:00:00+00:00,2,0
...,...,...,...
160133,2025-03-03 15:00:00+00:00,263,119
160134,2025-03-03 16:00:00+00:00,263,89
160135,2025-03-03 17:00:00+00:00,263,114
160136,2025-03-03 18:00:00+00:00,263,116


In [128]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160138 entries, 0 to 166161
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         160138 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  160138 non-null  int32                  
 2   rides               160138 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.7 MB


In [129]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [130]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160138 entries, 0 to 166161
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         160138 non-null  datetime64[us]
 1   pickup_location_id  160138 non-null  int32         
 2   rides               160138 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.7 MB


In [131]:
print(ts_data["pickup_location_id"].value_counts())
print(ts_data.head()) 

pickup_location_id
217    638
183    638
166    638
192    638
230    638
      ... 
29     638
23     638
81     638
122    638
68     638
Name: count, Length: 251, dtype: int64
          pickup_hour  pickup_location_id  rides
0 2025-02-23 18:00:00                 217      3
1 2025-02-26 03:00:00                 261      0
2 2025-02-17 12:00:00                  65      3
3 2025-02-19 08:00:00                  69      0
4 2025-02-06 10:00:00                 155      1


In [157]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [158]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 00:22:21,943 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:22:21,948 INFO: Initializing external client
2025-03-06 00:22:21,949 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:22:22,996 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669
Fetching data from 2025-02-05 05:22:21.943907+00:00 to 2025-03-06 04:22:21.943907+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.40s) 


In [159]:
current_date

Timestamp('2025-03-06 05:22:21.943907+0000', tz='Etc/UTC')

In [160]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,217,2025-02-27 00:00:00
1,0,0,0,0,0,0,0,0,0,0,...,16,12,1,8,2,2,35,11,261,2025-02-09 19:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,6,6,0,0,1,1,0,65,2025-02-10 23:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,2,0,69,2025-02-09 13:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,155,2025-02-22 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,5,2,0,3,49,2025-02-24 16:00:00
247,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,1,0,0,1,56,2025-02-18 07:00:00
248,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,202,2025-02-16 22:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,153,2025-02-06 07:00:00


In [161]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 00:25:18,214 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:25:18,224 INFO: Initializing external client
2025-03-06 00:25:18,224 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:25:19,080 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669


In [162]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [163]:
predictions

,pickup_location_id,predicted_demand
0,217,0.0
1,261,5.0
2,65,1.0
3,69,1.0
4,155,0.0
...,...,...
246,49,1.0
247,56,0.0
248,202,0.0
249,153,0.0


In [164]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
232,237,277.0
24,138,206.0
129,132,205.0
161,230,185.0
78,161,182.0
113,234,155.0
35,239,145.0
31,164,142.0
25,186,129.0
153,249,123.0
